In [70]:
import os
import time
import numpy as np
from rich.pretty import pprint
from typing import List, Dict, Tuple

from pycomex.functional.experiment import Experiment

In [71]:
PATH = os.getcwd()
RESULTS_PATH = os.path.join(PATH, 'results')

# A list of identifier strings that will be matched to decide which experiments to include 
# in the evaluation. These strings can be used to differentiate between different runs of 
# an experiment.
IDENTIFIERS: List[str] = ['hyperopt_b']

# These lists define which of the methods within the different categories should be included 
# in the computation to determine the best value of the hyperparameters.
BASE_METHOD_MAP: Dict[str, List[str]] = {
    'gnn': ['gatv2', 'gin'],
    'hdc': ['neural_net', 'random_forest'],
    'fp': ['neural_net', 'random_forest']
}

BASE_HPARAMS_MAP: Dict[str, List[str]] = {
    'gnn': ['BATCH_SIZE', 'LEARNING_RATE'],
    'fp': ['FINGERPRINT_SIZE', 'FINGERPRINT_RADIUS'],
    'hdc': ['EMBEDDING_SIZE', 'NUM_LAYERS']
}

METRIC: str = 'f1'
def get_metric(metrics: dict):
    #return 1 - metrics[METRIC]
    return metrics[METRIC]

In [72]:
# ~ finding experiments

print('traversing experiment namespaces...')
experiment_namespace_paths: List[str] = [
    path
    for file in os.listdir(RESULTS_PATH)
    if os.path.isdir(path := os.path.join(RESULTS_PATH, file))
]
pprint(experiment_namespace_paths)

print('traversing experiment paths...')
experiment_paths: List[str] = [
    path
    for folder_path in experiment_namespace_paths
    for file in os.listdir(folder_path)
    if os.path.isdir(path := os.path.join(folder_path, file))
]
pprint(experiment_paths, max_length=10)

traversing experiment namespaces...


[
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__aqsoldb',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__conjugated',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__qm9_smiles',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__bace',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__ames',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__qm9_smiles',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__clogp',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__bace',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__bbbp',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__qm9_smiles',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__clogp',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__aqsoldb',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__bbbp',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__conjugated',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__ames',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__conjugated',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__aqsoldb',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__conj',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__bbbp',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__hdc__ames',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__bace'
]

traversing experiment paths...


[
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__15_27__xfya',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__12_52__4QZd',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__15_07__sxCW',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__13_33__dsLN',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__13_41__WB07',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__15_17__eNpF',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__13_29__nLly',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__14_07__L8kH',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__12_49__Rhmh',
│   '/media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__clogp/ex_04_b__14_01_2025__14_42__zJ0B',
│   ... +843
]

In [73]:
import json

# ~ loading experiments
# Now that we have the paths to all the experiment archive folders, we can now actually 
# load them back into memory
experiments: List[Experiment] = []

time_start = time.time()
for path in experiment_paths:
    
    experiment_data_path = os.path.join(path, 'experiment_data.json')
    if not os.path.exists(experiment_data_path):
        print(f'no experiment data found at {experiment_data_path}')
        continue
    
    experiment_meta_path = os.path.join(path, 'experiment_meta.json')
    with open(experiment_meta_path, 'r') as file:
        metadata: dict = json.loads(file.read())
        parameters: dict = metadata['parameters']
    
    # We want to match a specific identifier
    if parameters['IDENTIFIER']['value'] not in IDENTIFIERS:
        continue
    
    experiment = Experiment.load(path)
    experiments.append(experiment)
    
time_end = time.time()
    
print(f'loaded {len(experiments)} experiments in {time_end - time_start:.1f} seconds')

no experiment data found at /media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__aqsoldb/debug/experiment_data.json
no experiment data found at /media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__conjugated/hyperopt_b__16_01_2025__08_56__Cv1n/experiment_data.json
no experiment data found at /media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__fp__ames/debug/experiment_data.json
no experiment data found at /media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__qm9_smiles/ex_01_a__10_01_2025__20_18__l2MR/experiment_data.json
no experiment data found at /media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/results/predict_molecules__gnn__qm9_smiles/ex_01_a__10_01_2025__21_18__xmQY/experiment_data.json
no experiment data found at /media/ssd/Programming/graph_hdc/graph_hdc/experiments/fingerprints/r

In [74]:
print('example experiment data:')
example_experiment = experiments[0]
pprint(experiments[0].data, max_length=12)

example experiment data:


{
│   'indices': {
│   │   'train': [9335, 5746, 5145, 9583, 7489, 9880, 9439, 7435, 5606, 8225, 5499, 2382, ... +8788],
│   │   'val': [733, 6393, 8325, 112, 3443, 5448, 98, 9848, 2186, 5506],
│   │   'test': [2201, 9327, 1033, 4179, 1931, 8119, 7366, 7739, 6219, 3439, 1537, 7995, ... +976]
│   },
│   'train_time': {
│   │   'random_forest': 1.2246356010437012,
│   │   'grad_boost': 6.266566753387451,
│   │   'neural_net': 65.70409560203552,
│   │   'support_vector': 120.1247251033783
│   },
│   'metrics': {
│   │   'test_random_forest': {
│   │   │   'acc': 0.9433198380566802,
│   │   │   'f1': 0.48541666666666666,
│   │   │   'ap': 0.05668016194331984,
│   │   │   'log_loss': 0.1739153223122866,
│   │   │   'mse': 0.04760342360793367
│   │   },
│   │   'test_grad_boost': {
│   │   │   'acc': 0.9453441295546559,
│   │   │   'f1': 0.5204055949084895,
│   │   │   'ap': 0.09037015615962984,
│   │   │   'log_loss': 0.1629787151995279,
│   │   │   'mse': 0.04192947098372066
│   │   },
│   │   'test_k_neighbors': {
│   │   │   'acc': 0.9473684210526315,
│   │   │   'f1': 0.7795967395967396,
│   │   │   'ap': 0.36846546310832023,
│   │   │   'log_loss': 0.5175489185358629,
│   │   │   'mse': 0.03981106612685558
│   │   },
│   │   'test_neural_net': {
│   │   │   'acc': 0.97165991902834,
│   │   │   'f1': 0.857905982905983,
│   │   │   'ap': 0.5554210044341623,
│   │   │   'log_loss': 0.09491917842095607,
│   │   │   'mse': 0.02197874593401418
│   │   },
│   │   'test_support_vector': {
│   │   │   'acc': 0.9676113360323887,
│   │   │   'f1': 0.821285639993669,
│   │   │   'ap': 0.4769013880855986,
│   │   │   'log_loss': 0.16274284301428973,
│   │   │   'mse': 0.03726976083449034
│   │   }
│   }
}

In [75]:
from collections import defaultdict

# Here we first group the experiment instances by the method that was used (aka the model)
# the model identifiers will be the keys and the values will be lists of experiments.
method_experiments_map: Dict[Tuple[str, str], List[Experiment]] = defaultdict(list)

for experiment in experiments:
    
    _, base, _ = experiment.metadata['name'].strip('.py').split('__')
    methods: List[str] = BASE_METHOD_MAP[base]
    for method in methods:
        method_experiments_map[(base, method)].append(experiment)

pprint(method_experiments_map, max_length=5)    

defaultdict(<class 'list'>, {
│   ('fp', 'neural_net'): [
│   │   <pycomex.functional.experiment.Experiment object at 0x71af4c2895d0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f21d0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f3a90>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f3160>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f0160>,
│   │   ... +34
│   ],
│   ('fp', 'random_forest'): [
│   │   <pycomex.functional.experiment.Experiment object at 0x71af4c2895d0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f21d0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f3a90>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f3160>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e4f0160>,
│   │   ... +34
│   ],
│   ('hdc', 'neural_net'): [
│   │   <pycomex.functional.experiment.Experiment object at 0x71af43f6b220>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af468c0790>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af43f69510>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af46d04880>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af46d05e70>,
│   │   ... +31
│   ],
│   ('hdc', 'random_forest'): [
│   │   <pycomex.functional.experiment.Experiment object at 0x71af43f6b220>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af468c0790>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af43f69510>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af46d04880>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af46d05e70>,
│   │   ... +31
│   ],
│   ('gnn', 'gatv2'): [
│   │   <pycomex.functional.experiment.Experiment object at 0x71af2e1374f0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af46ea0160>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af405d5a80>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af46ea2fe0>,
│   │   <pycomex.functional.experiment.Experiment object at 0x71af405d6e30>,
│   │   ... +31
│   ],
│   ... +1
})

In [76]:
for base, methods in BASE_METHOD_MAP.items():
    
    print(f'base: {base}')
    hparam_results_map: Dict[tuple, list[float]] = defaultdict(list)
    
    for experiment in experiments:
        
        _, base_, _ = experiment.metadata['name'].strip('.py').split('__')
        if base_ == base:
            hparams: tuple = tuple([experiment.parameters[hparam] for hparam in BASE_HPARAMS_MAP[base]])
            
            values: List[float] = []
            keys: List[str] = [f'test_{method}' for method in BASE_METHOD_MAP[base]]
            for key in keys:
                if 'metrics' in experiment.data:
                    value = get_metric(experiment.data['metrics'][key])
                    values.append(value)
                
            # Here we actually want to include the MEAN of all the different methods values as the indicator
            # to determine how well the hyperparameters performed
            hparam_results_map[hparams].append(np.mean(values))
            
    # Currently the dict data structure still contains lists as values which represent the different 
    # independent repetitions of the experiment. But we need to decide based on a single value 
    # which hyperparameters are the best. We can do this by taking the mean of all the different
    # results for each hyperparameter configuration.
    hparam_means_map = {hparam: np.mean(results) for hparam, results in hparam_results_map.items()}
    pprint(hparam_means_map)
    
    # Now we can sort the hyperparameters based on their mean value and select the best one
    if hparam_means_map:
        best_hparam = max(hparam_means_map.items(), key=lambda x: x[1])
        print(f'best hyperparameters: {best_hparam}')

base: gnn


{
│   (16, 1e-05): 0.8914190921297971,
│   (128, 0.001): 0.8871783666368156,
│   (64, 1e-05): 0.8349155511893255,
│   (16, 0.001): 0.9575757782123983,
│   (64, 0.001): 0.8867012848308681,
│   (16, 0.0001): 0.8740157122113503,
│   (32, 1e-05): 0.8350511298205578,
│   (32, 0.001): 0.8816885581201271,
│   (128, 0.0001): 0.8786320244942708,
│   (64, 0.0001): 0.8650032125155175,
│   (32, 0.0001): 0.9182033130662116,
│   (128, 1e-05): 0.755911894090341
}

best hyperparameters: ((16, 0.001), 0.9575757782123983)
base: hdc


{
│   (1024, 2): 0.821566645251233,
│   (1024, 3): 0.8253393574461808,
│   (2048, 3): 0.833176716422046,
│   (4096, 3): 0.8423248126947079,
│   (2048, 2): 0.8447460462564204,
│   (8192, 1): 0.8747765737206095,
│   (1024, 1): 0.8580133169478649,
│   (4096, 1): 0.8583627940795107,
│   (2048, 1): 0.8720488323727663,
│   (4096, 2): 0.829775815630781,
│   (8192, 3): 0.8193637645859271,
│   (8192, 2): 0.8540083973042579
}

best hyperparameters: ((8192, 1), 0.8747765737206095)
base: fp


{
│   (8192, 2): 0.6801050151864304,
│   (1024, 3): 0.6678777620196504,
│   (4096, 1): 0.6887767668924858,
│   (8192, 3): 0.6576435586912752,
│   (2048, 2): 0.6689271303924249,
│   (1024, 1): 0.6818388825577903,
│   (2048, 1): 0.6784299528881778,
│   (1024, 2): 0.6692036068805477,
│   (4096, 2): 0.6800862286921973,
│   (8192, 1): 0.6943678186105804,
│   (4096, 3): 0.6606773527554984,
│   (2048, 3): 0.6605235531465161
}

best hyperparameters: ((8192, 1), 0.6943678186105804)
